# 🚀 Chatbot creation

In this notebook, we'll walk through the steps required to train our own GPT model on the dialogs dataset

The code is adapted from the excellent [GPT tutorial](https://keras.io/examples/generative/text_generation_with_miniature_gpt/) created by Apoorv Nandan available on the Keras website.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

2024-03-19 16:17:22.666929: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 16:17:22.952527: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 16:17:23.669186: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-19 16:17:23.669267: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 900 #5000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 50

## 1. Load the data <a name="load"></a>

In [3]:
# Load the full dataset
with open("data/dialogs.json") as json_data:
    dialog_data = json.load(json_data)

In [4]:
dialog_data[10]

{'user': 'where are you from', 'response': 'i exist in the digital world'}

In [5]:
dialog_data[0:2]

[{'user': 'hello', 'response': 'hi there'},
 {'user': 'how are you today', 'response': 'i am doing well'}]

In [6]:
# Filter the dataset
filtered_data = [
    "user : "
    + x["user"]
    + " : "
    + x["response"]
    for x in dialog_data
    if x["user"] is not None
    and x["response"] is not None
]

In [7]:
example = filtered_data[3]
print(example)

user : tell me a joke : why do not skeletons fight each other? they do not have the guts


In [8]:
filtered_data

['user : hello : hi there',
 'user : how are you today : i am doing well',
 'user : what is your favorite color : i do not have a favorite color',
 'user : tell me a joke : why do not skeletons fight each other? they do not have the guts',
 'user : i feel happy : that is great to hear',
 'user : what do you do for fun : i like to chat with people like you',
 'user : tell me about yourself : i am a chatbot programmed to assist users',
 'user : i am bored : let us find something interesting to do',
 'user : what time is it : i do not have the capability to tell time',
 'user : goodbye : farewell',
 'user : where are you from : i exist in the digital world',
 'user : do you have any siblings : i do not have family like humans do',
 'user : what is the meaning of life : the meaning of life is subjective and varies from person to person',
 'user : what is your purpose : my purpose is to assist and interact with users',
 'user : are you a robot : i am a chatbot, which is a form of artificial

## 2. Tokenize the data <a name="tokenize"></a>

In [9]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [10]:
# Display an example of a recipe
example_data = text_data[4]
example_data

'user : i feel happy : that is great to hear'

In [11]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

2024-03-19 16:17:25.507887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-19 16:17:25.508301: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-19 16:17:25.508376: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-19 16:17:25.508446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file

In [12]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [13]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [14]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: :
3: user
4: do
5: i
6: .
7: you
8: of
9: is


In [15]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  3   2   5  35 280   2  28   9 428  14 210   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]


## 3. Create the Training Set <a name="create"></a>

In [16]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [17]:
example_input_output = train_ds.take(1).get_single_element()

In [18]:
# Example Input
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  3,   2,   5,  74,  29, 233,   6,   2,  29, 233, 409,  18,  16,
       105,  19, 220,  16, 183,   6,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])>

In [19]:
# Example Output (shifted by one token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  2,   5,  74,  29, 233,   6,   2,  29, 233, 409,  18,  16, 105,
        19, 220,  16, 183,   6,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])>

## 5. Create the causal attention mask function <a name="causal"></a>

In [20]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## 6. Create a Transformer Block layer <a name="transformer"></a>

In [21]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

## 7. Create the Token and Position Embedding <a name="embedder"></a>

In [22]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

## 8. Build the Transformer model <a name="transformer_decoder"></a>

In [23]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [24]:
gpt.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        250880    
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  ((None, None, 256),      658688    
 merBlock)                    (None, 2, None, None))             
                                                                 
 dense_2 (Dense)             (None, None, 900)         231300    
                                                                 
Total params: 1,140,868
Trainable params: 1,140,868
Non-trainable params: 0
___________________________________________________

In [25]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

## 9. Train the Transformer <a name="train"></a>

In [26]:
# # Create a TextGenerator checkpoint
# class TextGenerator(callbacks.Callback):
#     def __init__(self, index_to_word, top_k=10):
#         self.index_to_word = index_to_word
#         self.word_to_index = {
#             word: index for index, word in enumerate(index_to_word)
#         }

#     def sample_from(self, probs, temperature):
#         probs = probs ** (1 / temperature)
#         probs = probs / np.sum(probs)
#         return np.random.choice(len(probs), p=probs), probs

#     def generate(self, start_prompt, max_tokens, temperature):
#         start_tokens = [
#             self.word_to_index.get(x, 1) for x in start_prompt.split()
#         ]
#         sample_token = None
#         info = []
#         while len(start_tokens) < max_tokens and sample_token != 0:
#             x = np.array([start_tokens])
#             y, att = self.model.predict(x, verbose=0)
#             sample_token, probs = self.sample_from(y[0][-1], temperature)
#             info.append(
#                 {
#                     "prompt": start_prompt,
#                     "word_probs": probs,
#                     "atts": att[0, :, -1, :],
#                 }
#             )
#             start_tokens.append(sample_token)
#             start_prompt = start_prompt + " " + self.index_to_word[sample_token]
#         print(f"\ngenerated text:\n{start_prompt}\n")
#         return info

#     def on_epoch_end(self, epoch, logs=None):
#         self.generate("response", max_tokens=MAX_LEN, temperature=1.0)

In [27]:
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, user_input, max_tokens, temperature):
        start_prompt = "user : " + user_input + " : "  # Add ":" after user input
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        bot_response = ""  # Initialize bot's response
        info = []
        generated_response = False  # Flag to track if a response has been generated
        n = 0
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, _ = self.sample_from(y[0][-1], temperature)
            word = self.index_to_word[sample_token] if sample_token != 0 else ''
            bot_response += ' ' + word  # Add space between words
            start_tokens.append(sample_token)
            # generated_response = True  # Set flag to indicate that a response has been generated
        # Print user input with punctuation
        print(f"user: {user_input}")
        # Print bot's response
        # Extract the last part before ":" in the generated response
        bot_response_parts = bot_response.split(":")
        if len(bot_response_parts) > 1:
            bot_response = bot_response_parts[-1].strip()
        # Print bot's response
        print(f"bot: {bot_response}\n")
        return info

In [28]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [29]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/50
10/10 [==============================] - 2s 93ms/step - loss: 2.5631 - dense_2_loss: 2.5631
Epoch 2/50
10/10 [==============================] - 1s 84ms/step - loss: 1.2439 - dense_2_loss: 1.2439
Epoch 3/50
10/10 [==============================] - 1s 88ms/step - loss: 0.9711 - dense_2_loss: 0.9711
Epoch 4/50
10/10 [==============================] - 1s 87ms/step - loss: 0.7884 - dense_2_loss: 0.7884
Epoch 5/50
10/10 [==============================] - 1s 88ms/step - loss: 0.6512 - dense_2_loss: 0.6512
Epoch 6/50
10/10 [==============================] - 1s 84ms/step - loss: 0.5526 - dense_2_loss: 0.5526
Epoch 7/50
10/10 [==============================] - 1s 88ms/step - loss: 0.4771 - dense_2_loss: 0.4771
Epoch 8/50
10/10 [==============================] - 1s 90ms/step - loss: 0.4300 - dense_2_loss: 0.4300
Epoch 9/50
10/10 [==============================] - 1s 84ms/step - loss: 0.3812 - dense_2_loss: 0.3812
Epoch 10/50
10/10 [==============================] - 1s 91ms/step - loss:

In [30]:
# Save the final model
gpt.save("./models/gpt")

INFO:tensorflow:Assets written to: ./models/gpt/assets


INFO:tensorflow:Assets written to: ./models/gpt/assets


# 3. Generate text using the Transformer

In [31]:
def print_probs(info, vocab, top_k=20):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [32]:
info = text_generator.generate(
    "I can't forget my ex.", max_tokens=MAX_LEN, temperature=1.0
)

user: I can't forget my ex.
bot:  i ' m doing with your heart , cherishing moments of love . 



In [37]:
info = text_generator.generate(
    "I can't connect with new culture of a foreign country.", max_tokens=MAX_LEN, temperature=1.0
)

user: I can't connect with new culture of a foreign country.
bot:  in spirit , they are with you , sharing every moment of joy and sorrow . 



In [34]:
info = text_generator.generate(
    "I miss my home.", max_tokens=MAX_LEN, temperature=0.7
)

user: I miss my home.
bot:  i am a chatbot programmed to assist users 



In [35]:
info = text_generator.generate(
    "I am worried about the war.", max_tokens=MAX_LEN, temperature=0.9
)

user: I am worried about the war.
bot:  what has keys but can not open locks ? a piano 

